In [1]:
from keras.applications.vgg16 import VGG16
from keras.models import Model
import numpy as np

model = VGG16(weights='imagenet', include_top=False)


Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.


In [2]:
layer_out = [layer.output for layer in model.layers]

In [14]:
from keras.layers import Conv2D, Conv2DTranspose, Concatenate

def TeranusNet(pretrained_weights = None, Vgg_pretrain = False, input_size = (224,224,3)):
    """
    Representation of TeranusNet
    """
    
    model = VGG16(weights = 'imagenet', include_top = False)
    
    ## Down Layers
    Down1 = model.get_layer('block1_conv2').output
    Down2 = model.get_layer('block2_conv2').output
    Down3 = model.get_layer('block3_conv3').output
    Down4 = model.get_layer('block4_conv3').output
    Down5 = model.get_layer('block5_conv3').output
    
    Middle = model.get_layer('block5_pool').output
    
    ## Up layers
    X = Conv2D(512,3, activation = 'relu')(Middle)
    X = Conv2DTranspose(256,(4,2), activation = 'relu')(X)
    X = Concatenate(axis = -1, name = 'concat_1')([X, Down5])
    
    X = Conv2D(256 + 512,3, activation = 'relu')(X)
    X = Conv2DTranspose(256,(4,2), activation = 'relu')(X)
    X = Concatenate(axis = -1, name = 'concat_2')([X, Down4])
    
    X = Conv2D(256 + 512,3, activation = 'relu')(X)
    X = Conv2DTranspose(256, (4,2), activation = 'relu')(X)
    X = Concatenate(axis = -1, name = 'concat_3')([X, Down3])
    
    X = Conv2D(512, 3, activation = 'relu')(X)
    X = Conv2DTranspose(64,(4,2), activation = 'relu')(X)
    X = Concatenate(axis=-1, name = 'concat_4')([X, Down2])
    
    X = Conv2D(64+128,3, activation = 'relu')(X)
    X = Conv2D(32,(4,2), activation = 'relu')(X)
    X = Concatenate(axis=-1, name = 'concat_5')([X,Down1])
    
    conv1 = Conv2D(32,3, name = 'conv-1', activation = 'relu')(X)
    out = Conv2D(1,3, activation = 'softmax',name = 'Out')(conv1)

    out_model = Model(input = model.input, output = out)
    
    if (pretrained_weights):
        out_model.load_weights(pretrained_weights)
    
    return out_model
    

In [15]:
net = TeranusNet()

C:\Users\Ryan_Siv\Anaconda3\lib\site-packages\ipykernel_launcher.py:43: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("Ou...)`


In [16]:
net.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, None, None, 6 1792        input_6[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, None, None, 6 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, None, None, 6 0           block1_conv2[0][0]               
__________________________________________________________________________________________________
block2_con

In [10]:
res = model.get_layer('block2_conv2').output

In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________